In [ ]:
# Converts Hasura schemas from _snake_case to camelCase
import sys
!{sys.executable} -m pip install pyhumps

import requests
import json
import pandas as pd
import humps
import inflect
import time

## Enter Parameters Here:
hasura_hostname = 'https://projectname.hasura.app' # stripping the /v1/graphql, etc.
admin_secret = ''
db_source = 'default'

In [ ]:
# Introspect > query_root
query = '''
query introspectionQueryRoot {
  __type(name:"query_root") {
    kind
    name
    fields {
      name
    }
  }
}
'''
query_root = requests.post(hasura_hostname+'/v1/graphql', json={'query': query}, headers={'x-hasura-admin-secret': admin_secret})
json_data = json.loads(query_root.text)
fields = pd.DataFrame(json_data['data']['__type']['fields'])

print('>>> All Fields')
display(fields)
## Stripping from the _aggregate field to try and target just new tables and views
fields = fields[fields['name'].str.endswith('_aggregate')]
fields['name'] = fields['name'].str.replace('_aggregate', '')

print('>>> Table and View-Based Fields')
display(fields)

In [ ]:
# Inflect word to plural / singular
p = inflect.engine()

for i in fields.index:
  # De-camel any pre-camelized names
  field_decamel = humps.decamelize(fields.at[i,'name'])

  # Split up words to lists
  words = field_decamel.split('_')

  # Plural version - last word plural
  plural = p.plural_noun(words[-1])
  if plural != False:
    plural_words = words[:-1]
    plural_words.append(plural)
  else:
    plural_words = words

  # Singular version - last word singular
  singular = p.singular_noun(words[-1])
  if singular != False:
    singular_words = words[:-1]
    singular_words.append(singular)
  else:
    singular_words = words

  # Camel-ise lists
  plural_camel = plural_words[0] + ''.join(x.title() for x in plural_words[1:])
  singular_camel = singular_words[0] + ''.join(x.title() for x in singular_words[1:])

  # Build Object
  jsondata = {}
  args={}
  configuration={}
  custom_root_fields={}

  # Create Custom Root Field Payload
  jsondata['type'] = 'pg_set_table_customization'
  args['table'] = fields.at[i,'name']
  args['source'] = db_source
  configuration['identifier'] = singular_camel
  custom_root_fields['select'] = plural_camel
  custom_root_fields['select_by_pk'] = singular_camel
  custom_root_fields['select_aggregate'] = plural_camel + 'Aggregate'
  custom_root_fields['insert'] = plural_camel + 'Insert'
  custom_root_fields['insert_one'] = singular_camel + 'Insert'
  custom_root_fields['update'] = plural_camel + 'Update'
  custom_root_fields['update_by_pk'] = singular_camel + 'Update'
  custom_root_fields['delete'] = plural_camel + 'Delete'
  custom_root_fields['delete_by_pk'] = singular_camel + 'Delete'

  configuration['custom_root_fields'] = custom_root_fields
  args['configuration'] = configuration
  jsondata['args'] = args

  update_root_fields = requests.post(hasura_hostname+'/v1/metadata', json=jsondata, headers={'x-hasura-admin-secret': admin_secret})
  if update_root_fields.status_code == 200:
      print('-----------------------------------------')
      print('Success: ' + fields.at[i,'name'])
  else:
      print('-----------------------------------------')
      print('Failure: ' + fields.at[i,'name'])
      print(update_root_fields.reason)
  time.sleep(2) # Cooldown